In [25]:
import pandas as pd
from datetime import datetime, timedelta

## Clientes Normais ultimos 15 dias :

In [26]:
df =  pd.read_excel(r"N:\Drives compartilhados\22 - BI\99.Backup\BI Operações\Logística\OTIF\Cobertura.xlsx")

In [ ]:
df.head(3)

In [28]:
df1 = df[['Nome do Destinatário','Nota Fiscal','Previsão Entrega Cliente','Situação','CEP do destinatário','Status Transportador', 'e-mail Destinatário']]

In [ ]:
df1['Nota Fiscal'] = df1['Nota Fiscal'].astype(str)
df1['Previsão Entrega Cliente'] = pd.to_datetime(df1['Previsão Entrega Cliente'], errors='coerce', format='%d/%m/%Y')

In [ ]:
df1 = df1[(df1['Situação'] == "No prazo") & (df1['Status Transportador'] == 'Entregue')]
df1.info()

In [31]:
# Define a data de 12 meses atrás
hoje = datetime.today()
ultimos_30_dias = hoje - timedelta(days=15)

# Filtra as linhas onde a data na coluna 'Previsão Entrega Cliente' está dentro do intervalo
df_1_mes_atraz = df1[df1['Previsão Entrega Cliente'] >= ultimos_30_dias]


In [ ]:
# Define a data de 12 meses atrás e 1 mês atrás
hoje = datetime.today()
data_inicio = hoje - timedelta(days=365)  # 12 meses atrás
data_fim = hoje - timedelta(days=15)  # 1 mês atrás

# Filtra as linhas onde a data está entre 12 meses atrás e 1 mês atrás
df_ultimos_11_meses = df1[(df1['Previsão Entrega Cliente'] >= data_inicio) & (df1['Previsão Entrega Cliente'] < data_fim)]
df_ultimos_11_meses.shape


In [ ]:
# Filtra as linhas de df_1_mes_atraz onde o 'Nome do Destinatário' não está presente em df_ultimos_11_meses
df_diferenca = df_1_mes_atraz[~df_1_mes_atraz['e-mail Destinatário'].isin(df_ultimos_11_meses['e-mail Destinatário'])]

# Exibe o dataframe completo com os destinatários únicos em df_1_mes_atraz
df_diferenca.head(5)


In [ ]:
df_diferenca['Nota Fiscal'] = df_diferenca['Nota Fiscal'].astype(float).astype(int)

In [ ]:
mestre = pd.read_parquet(r'N:\Drives compartilhados\28 - Dados BI\2 Dados Transformados\2. Vendas\2.NFs_Venda.parquet')
mestre = mestre[mestre['Fat_Somado_n_considera'].isna()]

df_columns = mestre[['Data de Emissão','Núm. NF-e', 'BU Produtos']]
df_nc = df_columns.sort_values(by='Data de Emissão', ascending=False)
df_nc = df_columns[df_columns['BU Produtos'] =='NC']
df_nc.rename(columns={"Núm. NF-e": "Nota Fiscal"}, inplace=True)
df_nc.head(3)

In [ ]:
df_nc = df_nc.drop_duplicates(subset=['Nota Fiscal'])
df_nc.head(5)


In [ ]:
data_limite = pd.to_datetime('01/01/2023', format='%d/%m/%Y')
df_nc = df_nc[pd.to_datetime(df_nc['Data de Emissão'], format='%d/%m/%Y') > data_limite]
df_nc.shape

In [ ]:
df_diferenca.columns

In [39]:
df_nc['Nota Fiscal'] = df_nc['Nota Fiscal'].astype(str).astype(int)

In [ ]:
df_merged = pd.merge(df_diferenca, df_nc, on='Nota Fiscal', how='left')
df_merged.head(3)

In [ ]:
wpp = pd.read_parquet(r'N:\Drives compartilhados\28 - Dados BI\0 Dados Brutos\0. Neppo\0. Clientes_WhatsApp\0.Clientes_WhatsApp.parquet')

wpp_columns = wpp[['E-mail','Nome_Contato', 'Celular']]

wpp_columns.rename(columns={"E-mail":"e-mail Destinatário"}, inplace=True)

In [ ]:
wpp_columns[wpp_columns['e-mail Destinatário'] == 'oeliene54@gmail.com']

In [ ]:
df1[df1['e-mail Destinatário'] == 'oeliene54@gmail.com'].head(10)

In [ ]:
df_merged = pd.merge(df_merged, wpp_columns, on='e-mail Destinatário', how='left')
df_merged.head(3)

In [45]:
df_merged.drop_duplicates(subset=['Nota Fiscal'], inplace=True)


In [ ]:
df_merged.head(20)

In [ ]:
df_charge = df_merged[df_merged['BU Produtos'].notnull()]
df_resto = df_merged[df_merged['BU Produtos'].isnull()]
df_resto

In [48]:
df_charge.to_excel(r"N:\Drives compartilhados\NEO\05 - Marketing\01 - Time Marketing\10. Thyeres\Automações\Clientes_No_Prazo\Clientes_no_prazo_NC.xlsx", index=False)
df_resto.to_excel(r"N:\Drives compartilhados\NEO\05 - Marketing\01 - Time Marketing\10. Thyeres\Automações\Clientes_No_Prazo\Clientes_no_prazo.xlsx", index=False)